*Accompanying code examples of the book "Introduction to Artificial Neural Networks and Deep Learning: A Practical Guide with Applications in Python" by [Sebastian Raschka](https://sebastianraschka.com). All code examples are released under the [MIT license](https://github.com/rasbt/deep-learning-book/blob/master/LICENSE). If you find this content useful, please consider supporting the work by buying a [copy of the book](https://leanpub.com/ann-and-deeplearning).*
  
Other code examples and content are available on [GitHub](https://github.com/rasbt/deep-learning-book). The PDF and ebook versions of the book are available through [Leanpub](https://leanpub.com/ann-and-deeplearning).

In [1]:
#load watermark
%load_ext watermark
%watermark -a 'Gopala KR' -u -d -v -p watermark,numpy,matplotlib,nltk,sklearn,tensorflow,theano,mxnet,chainer,seaborn,keras,tflearn,bokeh,gensim

/srv/venv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
Using TensorFlow backend.
/srv/venv/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))


Gopala KR 
last updated: 2018-02-28 

CPython 3.6.3
IPython 6.2.1

watermark 1.6.0
numpy 1.14.1
matplotlib 2.1.2
nltk 3.2.5
sklearn 0.19.1
tensorflow 1.5.0
theano 1.0.1
mxnet 1.1.0
chainer 3.4.0
seaborn 0.8.1
keras 2.1.4
tflearn n
bokeh 0.12.14
gensim 3.3.0


/srv/venv/lib/python3.6/site-packages/scipy/sparse/sparsetools.py:20: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


# Model Zoo -- Softmax Regression

Implementation of softmax regression (multinomial logistic regression).

In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data


##########################
### DATASET
##########################

mnist = input_data.read_data_sets("./", one_hot=True)


##########################
### SETTINGS
##########################

# Hyperparameters
learning_rate = 0.5
training_epochs = 30
batch_size = 256

# Architecture
n_features = 784
n_classes = 10


##########################
### GRAPH DEFINITION
##########################

g = tf.Graph()
with g.as_default():

    # Input data
    tf_x = tf.placeholder(tf.float32, [None, n_features])
    tf_y = tf.placeholder(tf.float32, [None, n_classes])

    # Model parameters
    params = {
        'weights': tf.Variable(tf.zeros(shape=[n_features, n_classes],
                                               dtype=tf.float32), name='weights'),
        'bias': tf.Variable([[n_classes]], dtype=tf.float32, name='bias')}

    # Softmax regression
    linear = tf.matmul(tf_x, params['weights']) + params['bias']
    pred_proba = tf.nn.softmax(linear, name='predict_probas')
    
    # Loss and optimizer
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
        logits=linear, labels=tf_y), name='cost')
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    train = optimizer.minimize(cost, name='train')

    # Class prediction
    pred_labels = tf.argmax(pred_proba, 1, name='predict_labels')
    correct_prediction = tf.equal(tf.argmax(tf_y, 1), pred_labels)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='accuracy')

    
##########################
### TRAINING & EVALUATION
##########################

with tf.Session(graph=g) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = mnist.train.num_examples // batch_size

        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            _, c = sess.run(['train', 'cost:0'], feed_dict={tf_x: batch_x,
                                                            tf_y: batch_y})
            avg_cost += c
        
        train_acc = sess.run('accuracy:0', feed_dict={tf_x: mnist.train.images,
                                                      tf_y: mnist.train.labels})
        valid_acc = sess.run('accuracy:0', feed_dict={tf_x: mnist.validation.images,
                                                      tf_y: mnist.validation.labels})  
        
        print("Epoch: %03d | AvgCost: %.3f" % (epoch + 1, avg_cost / (i + 1)), end="")
        print(" | Train/Valid ACC: %.3f/%.3f" % (train_acc, valid_acc))
        
    test_acc = sess.run(accuracy, feed_dict={tf_x: mnist.test.images,
                                             tf_y: mnist.test.labels})
    print('Test ACC: %.3f' % test_acc)

Extracting ./train-images-idx3-ubyte.gz
Extracting ./train-labels-idx1-ubyte.gz
Extracting ./t10k-images-idx3-ubyte.gz
Extracting ./t10k-labels-idx1-ubyte.gz


/srv/venv/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



/srv/venv/lib/python3.6/site-packages/tensorflow/python/framework/tensor_util.py:539: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  return np.fromstring(tensor.tensor_content, dtype=dtype).reshape(shape)


Epoch: 001 | AvgCost: 0.476 | Train/Valid ACC: 0.900/0.907
Epoch: 002 | AvgCost: 0.341 | Train/Valid ACC: 0.910/0.915
Epoch: 003 | AvgCost: 0.320 | Train/Valid ACC: 0.911/0.918
Epoch: 004 | AvgCost: 0.307 | Train/Valid ACC: 0.916/0.921
Epoch: 005 | AvgCost: 0.301 | Train/Valid ACC: 0.919/0.924
Epoch: 006 | AvgCost: 0.297 | Train/Valid ACC: 0.919/0.923
Epoch: 007 | AvgCost: 0.291 | Train/Valid ACC: 0.922/0.925
Epoch: 008 | AvgCost: 0.289 | Train/Valid ACC: 0.920/0.924
Epoch: 009 | AvgCost: 0.285 | Train/Valid ACC: 0.923/0.926
Epoch: 010 | AvgCost: 0.284 | Train/Valid ACC: 0.923/0.924
Epoch: 011 | AvgCost: 0.281 | Train/Valid ACC: 0.922/0.926
Epoch: 012 | AvgCost: 0.279 | Train/Valid ACC: 0.924/0.925
Epoch: 013 | AvgCost: 0.278 | Train/Valid ACC: 0.926/0.925
Epoch: 014 | AvgCost: 0.276 | Train/Valid ACC: 0.926/0.926
Epoch: 015 | AvgCost: 0.274 | Train/Valid ACC: 0.925/0.927
Epoch: 016 | AvgCost: 0.275 | Train/Valid ACC: 0.926/0.930
Epoch: 017 | AvgCost: 0.273 | Train/Valid ACC: 0.927/0.9